#  Download Latest Immigration Enforcement Data POC
A Python-based notebook that scrapes the latest monthly immigration enforcement report from the U.S. DHS OHSS website, automatically downloads the most recent `.xlsx` or `.csv` file, and loads the data into a pandas DataFrame.

In [ ]:
#Install required libraries
!pip install requests beautifulsoup4 pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [96]:
import requests, bs4, pandas as pd, openpyxl, sys

print("Libraries and Versions:")
print(f"Python: {sys.version}")
print(f"requests: {requests.__version__}")
print(f"beautifulsoup4: {bs4.__version__}")
print(f"pandas: {pd.__version__}")
print(f"openpyxl: {openpyxl.__version__}")

Libraries and Versions:
Python: 3.9.6 (default, Nov 11 2024, 03:15:38) 
[Clang 16.0.0 (clang-1600.0.26.6)]
requests: 2.32.4
beautifulsoup4: 4.13.4
pandas: 2.3.0+4.g1dfc98e16a
openpyxl: 3.1.5


In [84]:
from bs4 import BeautifulSoup

url = "https://ohss.dhs.gov/topics/immigration/immigration-enforcement/monthly-tables"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

# Extract table rows
rows = soup.select("table tr")
values = []  # List to store (date, link) tuplesS
print(rows)


[<tr>
<th id="view-title-table-column" role="columnheader" scope="col">Data File</th>
<th class="views-field views-field-field-file-2" id="view-field-file-2-table-column" role="columnheader" scope="col">File Extension</th>
<th class="views-field views-field-field-file-2" id="view-filesize-table-column" role="columnheader" scope="col">File size</th>
<th class="views-field views-field-field-file-2" id="view-field-date-posted-table-column" role="columnheader" scope="col">Date Posted</th>
<th class="views-field views-field-field-report-year" id="view-field-report-year-table-column" role="columnheader" scope="col">Report Year</th>
<th class="views-field views-field-field-data-categories" id="view-field-data-categories-table-column" role="columnheader" scope="col">Data Category</th>
<th class="views-field views-field-field-data-categories-1" id="view-field-data-categories-1-table-column" role="columnheader" scope="col">categories</th>
</tr>, <tr>
<td data-order="Immigration Enforcement and L

In [85]:
import re

for row in rows:
    link = row.find("a", href=True)

    if link and (link["href"].endswith(".xlsx") or link["href"].endswith(".csv")):
        text = row.get_text(strip=True)

        #regex to pull the exact "Month Year" string
        match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{4}",text, flags=re.IGNORECASE)

        if match:
            part = match.group()
            try:
                date = pd.to_datetime(part, format="%B %Y")
                values.append((date, link["href"]))
                print(f"Added: ({date}, {link['href']})")
            except Exception:
                pass
        else:
            print(f"No valid Month-Year found in: {text}")

Added: (2024-11-01 00:00:00, /sites/default/files/2025-01/2025_0116_ohss_immigration-enforcement-and-legal-processes-tables-november-2024.xlsx)
Added: (2024-08-01 00:00:00, /sites/default/files/2024-12/2024_1206_ohss_immigration-enforcement-and-legal-processes-tables-august-2024.xlsx)
Added: (2024-07-01 00:00:00, /sites/default/files/2024-11/2024_1108_ohss_immigration-enforcement-and-legal-processes-tables-july-2024.xlsx)
Added: (2024-06-01 00:00:00, /sites/default/files/2024-10/24-1011_ohss_immigration-enforcement-and-legal-processes-tables-june-2024_2.xlsx)
Added: (2024-05-01 00:00:00, /sites/default/files/2024-09/24-0906_ohss_immigration-enforcement-and-legal-processes-tables-may-2024_v2.xlsx)
Added: (2024-04-01 00:00:00, /sites/default/files/2024-08/24-0809_ohss_immigration-enforcement-and-legal-processes-tables-april-2024.xlsx)
Added: (2024-03-01 00:00:00, /sites/default/files/2024-07/24-0705_ohss_immigration-enforcement-and-legal-processes-tables-march-2024.xlsx)
Added: (2024-02-

In [ ]:
# function to extract date from values tuple
def get_date_from_tuple(value):
    return value[0]

# Get the most recent file link
latest = max(values, key=get_date_from_tuple)
latest_link = latest[1]

# Construct full URL
base_url = "https://ohss.dhs.gov"
latest_url = base_url + latest_link

print("Latest URL:", latest_url)

Latest URL: https://ohss.dhs.gov/sites/default/files/2025-01/2025_0116_ohss_immigration-enforcement-and-legal-processes-tables-november-2024.xlsx


In [94]:
# Determine the file type from the URL and download

file_type = latest_url.split('.')[-1].lower()

r = requests.get(latest_url)
with open(f"latest_file.{file_type}", "wb") as f:
    f.write(r.content)

#load it from the downloaded file
if file_type == "xlsx":
    df = pd.read_excel(f"latest_file.{file_type}")
elif file_type == "csv":
    df = pd.read_csv(f"latest_file.{file_type}")
else:
    raise ValueError(f"Unsupported file format: .{file_type}")

In [95]:
print("Shape:", df.shape)
df.head(10)

Shape: (40, 3)


,Table of Contents,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,Click link for corresponding tab:,NaN,NaN
2,NaN,NaN,NaN
3,Category,Table,Starting Date
4,Encounters,Nationwide CBP Encounters by Encounter Type an...,2014
5,Encounters,CBP SW Border Encounters by Agency and Selecte...,2014
6,Encounters,CBP SW Border Encounters by Agency and Family ...,2014
7,Encounters,CBP SW Border Encounters by Agency and Sector ...,2014
8,Encounters,Nationwide USBP Encounters by Top 100 Citizens...,2014
9,Encounters,Nationwide OFO Enforcement Encounters by Top 1...,2014
